# Labeling Functions


In [1]:
import re
import glob
import emoji
import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize
from nltk import ngrams
from nltk.corpus import stopwords
from sentence_transformers import util

from snorkel.preprocess import preprocessor
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel
from sklearn.metrics import classification_report

In [2]:
# defining data paths
prop_data_annotated = "../data/processed/propaganda_annotated.pkl"
gen_data_annotated = "../data/processed/genuine_annotated.pkl"
wanlp_prop_data_path = "../data/raw/task1_train.json"
weakly_labeled_data_path = "../data/processed/train.json"
lfs_analysis_path = "../data/processed/lfs_analysis.json"
lfs_dev_labels_path = "../data/processed/lfs_dev_labels.npy"
lfs_train_labels_path = "../data/processed/lfs_train_labels.npy"

# we need the lf_dev data to report the performance of LFs
# and then remove it with the test data from the unlabaled data
lf_dev_data_path = "../data/processed/lf_dev.json"
test_data_path = "../data/processed/test_data.json"

# defining lexicons paths
proppy_lex_paths = "../data/raw/proppy_lexicons/*.csv"
loaded_lex_path = "../data/raw/loaded-language-lexicons.csv"

# loading pre-trained encoders
fasttext_model_path = "../models/cc.ar.300.bin"

In [3]:
# defining the needed fields by the working labeling functions
# in case of doing the analysis from scratch, comment the variable

# fields = [
#     "tweetid",
#     "user_profile_description",
#     "tweet_text",
#     "is_retweet",
#     "quote_count",
#     "reply_count",
#     "like_count",
#     "retweet_count",
#     "hashtags",
#     "urls",
#     "user_mentions",
#     "text",
#     "emojis",
#     "word_count",
#     "docs",
#     "is_irony",
#     "is_hate",
#     "xlmroberta_label",
# ]

In [4]:
df1 = pd.read_pickle(prop_data_annotated)  # [fields]
df2 = pd.read_pickle(gen_data_annotated)  # [fields]

In [5]:
# the following lines handle date conversion

df1["account_creation_date"] = pd.to_datetime(df1.account_creation_date, unit="ms")
df2["account_creation_date"] = pd.to_datetime(df2.account_creation_date, unit="ns")

In [6]:
# combining the propaganda and genuine users data

unlabeled_data = pd.concat([df1, df2], ignore_index=True)
unlabeled_data = unlabeled_data.sample(frac=1.0).reset_index(drop=True)

In [7]:
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,544945186909220864,607579193,حمود الداخل,alntafat,"الرياض, المملكة العربية السعودية",مشينا على درب المعزه ونهج العود ومن مدرسه شيخ ...,61366.0,6869.0,2012-06-13 00:00:00,RT @axxa00: توكلت على الله ولا حول ولاقوة الا ...,...,1,0,1,توكلت على الله ولا حول ولاقوة الا بالله استغفر...,1,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9975724816...","{'label': 'not offensive', 'score': 0.99894624...",{'sequence': 'توكلت على الله ولا حول ولاقوة ال...
1,1117798852772401152.0,2673203522.0,None,None,None,None,NaN,NaN,NaT,RT @MOISaudiArabia: الأمير عبدالعزيز بن سعود و...,...,0,1,1,الأمير عبدالعزيز بن سعود وزير الداخلية يرعى حف...,0,13,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9980658888...","{'label': 'not offensive', 'score': 0.99932289...",{'sequence': 'الأمير عبدالعزيز بن سعود وزير ال...


In [8]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196611 entries, 0 to 196610
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   196609 non-null  object 
 1   userid                    196610 non-null  object 
 2   user_display_name         187650 non-null  object 
 3   user_screen_name          187650 non-null  object 
 4   user_reported_location    146204 non-null  object 
 5   user_profile_description  183083 non-null  object 
 6   follower_count            187650 non-null  float64
 7   following_count           187650 non-null  float64
 8   account_creation_date     187650 non-null  object 
 9   tweet_text                196611 non-null  object 
 10  is_retweet                196611 non-null  bool   
 11  quote_count               196606 non-null  float64
 12  reply_count               196607 non-null  float64
 13  like_count                196607 non-null  o

In [9]:
labeled = pd.read_json(lf_dev_data_path)
labeled.head(2)

,tweetid,tweet_text,text,tech,label
0,924924839902793728,RT @Amal_onzi: 🕊💕هُو جنْةبعِيني.,RT : هو جنةبعيني.,None,0
1,1074734231887187970,ر٣ #تركيا_تجاهر_بالمعاصي,ر٣ تركيا تجاهر بالمعاصي,smears - name-calling - loaded language,1


In [10]:
labeled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     500 non-null    int64 
 1   tweet_text  500 non-null    object
 2   text        500 non-null    object
 3   tech        48 non-null     object
 4   label       500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.4+ KB


In [11]:
test = pd.read_json(test_data_path)
test.head(2)

,tweetid,tweet_text,text,tech,label
0,977553193814122498,شاركوا معنا .. في #ساعة_الأرض الليلة \nساعة وا...,شاركوا معنا .. في ساعة الأرض الليلة ساعة واحد ...,None,0
1,1005856990436970497,RT @qtfcjohz: َاللهم طهر قلوبنا من كل ضيق \nوي...,RT : اللهم طهر قلوبنا من كل ضيق ويسر أمورنا في...,None,0


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 0 to 419
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     420 non-null    int64 
 1   tweet_text  420 non-null    object
 2   text        420 non-null    object
 3   tech        40 non-null     object
 4   label       420 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 19.7+ KB


In [13]:
labeled_data = unlabeled_data[unlabeled_data.tweetid.isin(labeled.tweetid)]
labeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
105,326832860319326208,291216268,Booooooo,_Keano16,None,مانشستر قصة حب كلما حاولت الإبتعاد عنها اقتربت...,92553.0,292.0,2011-05-01 00:00:00,اليوم قابلت هالمليح @SirM7md وجلست معه وتمنيت ...,...,0,0,1,اليوم قابلت هالمليح وجلست معه وتمنيت انها بوقت...,0,14,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9976009726...","{'label': 'not offensive', 'score': 0.99777406...",{'sequence': 'اليوم قابلت هالمليح وجلست معه وت...
143,1090627394254356480,444707291,هُدؤ عَآشِقْ,M6l8l,None,أگتب لأعيش وأعيش لـِ أقرأ أكتُب حتّى لآ تخنقني...,5481.0,3740.0,2011-12-23 00:00:00,اللهم ثبت قلوبنا على دينك\n#اكشتتت_معنا٣,...,1,0,0,اللهم ثبت قلوبنا على دينك اكشتتت معنا٣,0,7,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9975403547...","{'label': 'not offensive', 'score': 0.99853730...",{'sequence': 'اللهم ثبت قلوبنا على دينك اكشتتت...


In [14]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 105 to 195989
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   userid                    500 non-null    object 
 2   user_display_name         500 non-null    object 
 3   user_screen_name          500 non-null    object 
 4   user_reported_location    295 non-null    object 
 5   user_profile_description  462 non-null    object 
 6   follower_count            500 non-null    float64
 7   following_count           500 non-null    float64
 8   account_creation_date     500 non-null    object 
 9   tweet_text                500 non-null    object 
 10  is_retweet                500 non-null    bool   
 11  quote_count               500 non-null    float64
 12  reply_count               500 non-null    float64
 13  like_count                500 non-null    object 
 14  retweet_co

In [15]:
# removing the labeled tweets from the unlabeled data

unlabeled_data = unlabeled_data[~unlabeled_data.tweetid.isin(labeled_data.tweetid)]
unlabeled_data = unlabeled_data[~unlabeled_data.tweetid.isin(test.tweetid)]
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [16]:
labeled_data["text"] = labeled_data.text.apply(lambda x: x.replace("RT :", "").strip())
labeled_data = labeled_data.reset_index(drop=True)
labels = [labeled[labeled.tweetid == i].label.values[0] for i in labeled_data.tweetid]
labeled_data["label"] = labels

In [17]:
# making sure the dataset doesn't contain any null values

subset = ["tweetid", "text", "quote_count", "xlmroberta_label"]
unlabeled_data = unlabeled_data.dropna(subset=subset)
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [18]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195686 entries, 0 to 195685
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   195686 non-null  object 
 1   userid                    195686 non-null  object 
 2   user_display_name         186730 non-null  object 
 3   user_screen_name          186730 non-null  object 
 4   user_reported_location    145640 non-null  object 
 5   user_profile_description  182231 non-null  object 
 6   follower_count            186730 non-null  float64
 7   following_count           186730 non-null  float64
 8   account_creation_date     186730 non-null  object 
 9   tweet_text                195686 non-null  object 
 10  is_retweet                195686 non-null  bool   
 11  quote_count               195686 non-null  float64
 12  reply_count               195686 non-null  float64
 13  like_count                195686 non-null  o

In [19]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   userid                    500 non-null    object 
 2   user_display_name         500 non-null    object 
 3   user_screen_name          500 non-null    object 
 4   user_reported_location    295 non-null    object 
 5   user_profile_description  462 non-null    object 
 6   follower_count            500 non-null    float64
 7   following_count           500 non-null    float64
 8   account_creation_date     500 non-null    object 
 9   tweet_text                500 non-null    object 
 10  is_retweet                500 non-null    bool   
 11  quote_count               500 non-null    float64
 12  reply_count               500 non-null    float64
 13  like_count                500 non-null    object 
 14  retweet_co

### User LFs


In [20]:
prop = 1
gen = 0
ab = -1

In [21]:
@labeling_function()
def missing_bio(example):
    """Label all tweets of a user as propaganda if they don't have a bio."""
    if pd.isna(example.user_profile_description):
        return prop
    else:
        return ab

In [22]:
@labeling_function()
def missing_loc(example):
    """Label all tweets of a user as propaganda if they don't have a location."""
    if pd.isna(example.user_reported_location):
        return prop
    else:
        return ab

In [23]:
@labeling_function()
def created_2018_2019(example):
    """Label all tweets of a user as propaganda if account is created 2018 or 2019."""
    if example.account_creation_date.year in [2018, 2019]:
        return prop
    else:
        return ab

In [24]:
@labeling_function()
def created_2011_2012(example):
    """Label all tweets of a user as transparent if account is created 2011 or 2012."""
    if example.account_creation_date.year in [2011, 2012]:
        return gen
    else:
        return ab

In [25]:
@preprocessor(memoize=False)
def tokenize_bio(example):
    """Tokenize text in the bio."""
    if not pd.isna(example.user_profile_description):
        example.bio_tokens = word_tokenize(example.user_profile_description)
    else:
        example.bio_tokens = None
    return example

In [26]:
@labeling_function(pre=[tokenize_bio])
def bio_keywords(example):
    """Label all tweets of a user as transparent if bio contains certain lexicons."""
    keys = ["الحساب", "الرسمي", "عضو", "رئيس", "كاتب", "إدارة"]
    if example.bio_tokens is not None:
        if any(np.in1d(keys, example.bio_tokens)):
            return gen
        else:
            return ab
    else:
        return ab

In [27]:
@labeling_function()
def follow_ratio_prop(example):
    """Label all tweets of a user as transparent if 0.8 <= follow ratio <= 1.2"""
    num_followers = example.follower_count
    num_following = example.following_count
    ratio = num_followers / num_following if num_following != 0 else 0
    if 0.8 <= ratio <= 1.2:
        return prop
    else:
        return ab

In [28]:
@labeling_function()
def follow_ratio_gen(example):
    """Label all tweets of a user as transparent if follow ratio <= 0.2"""
    num_followers = example.follower_count
    num_following = example.following_count
    ratio = num_followers / num_following if num_following != 0 else 0
    if 0.0 <= ratio <= 0.2:
        return gen
    else:
        return ab

### Tweet LFs


In [29]:
@labeling_function()
def contain_url(example):
    """Label tweet as transparent if it contains a URL."""
    if example.urls > 0:
        return gen
    else:
        return ab

In [30]:
@labeling_function()
def contain_mention(example):
    """Label tweet as propaganda if it contains a mention."""
    if example.user_mentions > 0:
        return prop
    else:
        return ab

In [31]:
@labeling_function()
def labeling_sarcasm(example):
    """Label tweet as propaganda if it contains the name calling tech (sarcasm)."""
    if len(example.docs.entities) and example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [32]:
@labeling_function()
def labeling_hate(example):
    """Label tweet as propaganda if it contains the name calling tech (hate)."""
    if len(example.docs.entities) and example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [33]:
@labeling_function()
def contain_ent(example):
    """Label tweet as propaganda if it contains an entity."""
    if len(example.docs.entities):
        return prop
    else:
        return ab

In [34]:
@labeling_function()
def ent_free(example):
    """Label tweet as transparent if it doesn't contain any entities."""
    if len(example.docs.entities):
        return ab
    else:
        return gen

In [35]:
@labeling_function()
def contain_question(example):
    """Label tweet as propaganda if it contains an question."""
    for w in example.docs.sentences[0].words:
        if w.upos == "AUX":
            return prop
    return ab

In [36]:
# loading the manually crafted loaded tokens.
# extracted only from the 500 tweets used for labeling functions development.

loaded_lexicons = pd.read_csv(loaded_lex_path)["loaded-language"].to_list()
loaded_lexicons[:5]

['جاهر', 'جهر', 'تجاهر بالمعاصي', 'مجاهرة بالمعاصي', 'تجاهر بالمعاصى']

In [37]:
# loading the proppy lexicons introduced in https://arxiv.org/abs/1912.06810

# proppy_lexicons = []
# for file in glob.glob(proppy_lex_paths):
#     with open(file, encoding="utf-8") as f:
#         proppy_lexicons.extend(f.readlines())

proppy_lexicons = pd.read_csv(glob.glob(proppy_lex_paths)[0])
proppy_lexicons.head(2)

,factives,hedges,implicatives,report_verbs,bias,negative_words,positive_words,negative_colloquial_words,positive_colloquial_words
0,عارف,واضح,يدير,يتهم,إسقاط,أبو وجهين,فالح,مزوره,متحمس
1,يدري,فيما يبدو,يتذكر,اتهم,إجهاض,بوجهين,شاطر,مغترب,حماس


In [38]:
# removing diacritization from proppy lexicons

# pattern = r"[\u0617-\u061A\u064B-\u0652]"
# proppy_lexicons = [re.sub(pattern, "", term.strip()) for term in proppy_lexicons]

In [39]:
proppy_lexicons = proppy_lexicons.to_dict(orient="list")

In [40]:
proppy_lexicons = {k: [w for w in v if not pd.isna(w)] for k, v in proppy_lexicons.items()}

In [41]:
print("Number of words in each proppy lexicon type ...")
print([len(v) for v in proppy_lexicons.values()])

Number of words in each proppy lexicon type ...
[42, 109, 43, 212, 578, 4630, 1950, 1468, 914]


In [42]:
# combining all the proppy lexicons

proppy = []
for plex in proppy_lexicons:
    proppy.extend(plex)

In [43]:
# we are removing diacritization from stanza lemmas


@preprocessor(memoize=False)
def tokenize_tweet(example):
    """Tokenize and lemmatize text in tweets."""
    if len(example.docs.sentences):
        example.tweet_tokens = [w.text for w in example.docs.sentences[0].words]
        example.tweet_lemmas = [re.sub(pattern, "", w.lemma) for w in example.docs.sentences[0].words]
    else:
        example.tweet_tokens = []
        example.tweet_lemmas = []
    return example

In [44]:
@preprocessor(pre=[tokenize_tweet], memoize=False)
def bigram_tweet(example):
    """Create bigrams of tweet text's tokens and lemmas."""
    example.bigram_tokens = [
        " ".join(gram) for gram in ngrams(example.tweet_tokens, 2) if len(gram)
    ]
    example.bigram_lemmas = [
        " ".join(gram) for gram in ngrams(example.tweet_lemmas, 2) if len(gram)
    ]
    return example

In [45]:
@labeling_function(pre=[bigram_tweet])
def loaded_language(example):
    """Label tweet as propaganda if it contains any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) >= 1:
        return prop
    else:
        return ab

In [46]:
@labeling_function(pre=[bigram_tweet])  # [bigram_tweet]
def genuine_language(example):
    """Label tweet as transparent if it doesn't contain any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) == 0:
        return gen
    else:
        return ab

In [47]:
@labeling_function(pre=[bigram_tweet])
def loaded_proppy(example):
    """Label tweet as propaganda if it contains any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy)) >= 1:
        return prop
    else:
        return ab

In [48]:
@labeling_function(pre=[bigram_tweet])
def genuine_proppy(example):
    """Label tweet as transparent if it doesn't contain any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy)) == 0:
        return gen
    else:
        return ab

The following commented codes are to split each proppy lexicon type.

In [49]:
name_space = {}
proppy_lfs = []

In [50]:
for name in proppy_lexicons.keys():
    code = f"""
@labeling_function(pre=[bigram_tweet])
def loaded_proppy_{name}(example):
    \"""Label tweet as propaganda if it contains any of the proppy lexicons.\"""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons["{name}"])) >= 1:
        return prop
    else:
        return ab
    """
    code = compile(code, "<string>", "exec")
    exec(code, globals(), name_space)
    proppy_lfs.append(name_space[f"loaded_proppy_{name}"])

In [51]:
for name in proppy_lexicons.keys():
    code = f"""
@labeling_function(pre=[bigram_tweet])
def genuine_proppy_{name}(example):
    \"""Label tweet as transparent if it doesn't contain any of the proppy lexicons.\"""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons["{name}"])) == 0:
        return gen
    else:
        return ab
    """
    code = compile(code, "<string>", "exec")
    exec(code, globals(), name_space)
    proppy_lfs.append(name_space[f"genuine_proppy_{name}"])

In [52]:
len(proppy_lfs)

18

In [53]:
@labeling_function()
def loaded_hate(example):
    """Label tweet as propaganda if it contains hate speech (loaded language)."""
    if example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [54]:
@labeling_function()
def loaded_sarcasm(example):
    """Label tweet as propaganda if it contains sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [55]:
@labeling_function()
def gen_hate(example):
    """Label tweet as transparent if it doesn't contain hate speech (loaded language)."""
    if example.hate["label"] == "not offensive":
        return gen
    else:
        return ab

In [56]:
@labeling_function()
def gen_sarcasm(example):
    """Label tweet as transparent if it doesn't contain sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Non-Sarcasm":
        return gen
    else:
        return ab

In [57]:
# flag_waving

flag_engine = re.compile(r"Number=\w+")


@labeling_function()
def flag_wave(example):
    """Label tweet as propaganda if it contains plural pronouns (flag-waving)."""
    for w in example.docs.sentences[0].words:
        if (
            w.upos == "PRON"
            and flag_engine.findall(w.feats)[0].split("=")[-1] == "Plur"
        ):
            return prop
    return ab

In [58]:
# loading the Arabic NLTK stop words.
arabic_stop_words = stopwords.words("arabic")

# removing diacritization from stop words
pattern = r"[\u0617-\u061A\u064B-\u0652]"
arabic_stop_words = [re.sub(pattern, "", w) for w in arabic_stop_words]

In [59]:
@labeling_function(pre=[tokenize_tweet])
def repetition(example):
    """Label tweet as propaganda if it has at least one repeated token."""
    tokens = [word for word in example.tweet_tokens if word not in arabic_stop_words]
    tokens = pd.Series(tokens)
    if tokens.value_counts().max() >= 2:
        return prop
    else:
        return ab

In [60]:
# loading the WANLP propaganda dataset for distant supervision
# dataset source: https://gitlab.com/araieval/propaganda-detection


wanlp_train = pd.read_json(wanlp_prop_data_path)

label_processing = lambda x: 0 if "no technique" in x else 1
wanlp_train["labels"] = wanlp_train["labels"].apply(label_processing)

wanlp_train.head(2)

,id,text,labels
0,1358824915483435008,"#بي_بي_سي_ترندينغ: النساء ""تثرثر كثيرا"" رئيس أ...",0
1,1389927866356412416,"""ده مش معتقل ده أحسن من اللوكاندة"".. جدل وسخري...",1


In [61]:
def clean_text(text):
    """Process text and remove links, symbols, and diacritization."""
    # links
    clean_text = re.sub(r"http\S+|t\.co/\S+", "", text)
    # mentions
    clean_text = re.sub(r"@\w+", "", clean_text)
    # hashtags
    clean_text = re.sub(r"#", "", clean_text)
    clean_text = re.sub(r"_", " ", clean_text)
    # tashqeel - from @bakriano
    clean_text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", clean_text)
    # emojis
    clean_text = emoji.replace_emoji(clean_text, replace="")
    # remove new lines and normalize white spaces
    clean_text = re.sub(r"\s+", " ", clean_text)
    return clean_text.replace("RT :", "").strip()


wanlp_train["text"] = wanlp_train["text"].apply(clean_text)

In [62]:
encoder = fasttext.load_model(fasttext_model_path)

wanlp_hidden_states = []
for tweet in tqdm(wanlp_train.text.values, total=len(wanlp_train)):
    vec = encoder.get_sentence_vector(tweet)
    wanlp_hidden_states.append(list(vec))

  0%|          | 0/504 [00:00<?, ?it/s]

In [63]:
wanlp_hidden_states = np.array(wanlp_hidden_states)
wanlp_hidden_states.shape

(504, 300)

In [62]:
@preprocessor()
def get_sim_scores(example):
    """Get similarity score between tweet and WANLP propaganda tweets."""
    tweet_vec = encoder.get_sentence_vector(example.text)
    sim_scores = util.cos_sim(tweet_vec, wanlp_hidden_states)
    example.sim_scores = sim_scores
    return example

In [63]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_prop(example):
    """Label tweet as propaganda based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 1
        and sim_scores[-1][most_sim].item() >= 0.75
    ):
        return prop
    else:
        return ab

In [64]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_gen(example):
    """Label tweet as transparent based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 0
        and sim_scores[-1][most_sim].item() >= 0.65
    ):
        return gen
    else:
        return ab

In [65]:
@labeling_function()
def slogans(example):
    """Label tweet as propaganda if it has any of the slogans form."""
    matches = re.findall(r"لا ل\w+", example.text)
    matches += re.findall(r"نعم ل\w+", example.text)
    matches += re.findall(r"لا بديل", example.text)
    if len(matches):
        return prop
    else:
        return ab

In [66]:
# defining the manually extracted hated organizations and entities.

hitlerum = [
    "هتلر",
    "البغدادي",
    "اوردوغان",
    "قطر",
    "داعش",
    "حوثي",
    "تركيا",
    "الشيعة",
    "إيران",
    "ايران",
    "اخونجي",
    "اخوان",
    "إخوان",
    "إخوانجي",
    "أوردوغان",
    "الحوثي",
    "الحوثيين",
    "ستالين",
    "الإخوان",
    "الاخوان",
    "إرهابي",
    "الإرهابيين",
    "متطرف",
    "المتطرفين",
    "شيعي",
]

In [67]:
@labeling_function(pre=[bigram_tweet])
def reductio(example):
    """Label tweet as propaganda if it contains the Reductio Ad Hitlerum tech."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if any(np.in1d(tweet_ngrams, hitlerum)):
        return prop
    else:
        return ab

In [68]:
@labeling_function()
def exaggeration(example):
    """Label tweet as propaganda if has the "أفعل" preference form."""
    if not len(example.docs.sentences):
        return ab
    for w in example.docs.sentences[0].words:
        if w.lemma.startswith("أ") and w.upos == "ADJ":
            return prop
    return ab

In [69]:
@labeling_function()
def pronouns(example):
    """Label tweet as propaganda if has at least one pronoun."""
    pro_nouns = []
    for word in example.docs.sentences[0].words:
        if word.upos == "PRON":
            pro_nouns.append(word.text)
    if len(pro_nouns) >= 1:
        return prop
    else:
        return ab

In [70]:
# defining the XLM-RoBERTa Zero-Shot model classes
candidate_labels = ["transparent", "propaganda"]
candidate_labels

['transparent', 'propaganda']

In [71]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_prop(example):
    """Label tweet as propaganda based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.90:
        if example.xlmroberta_label["labels"][0] == candidate_labels[-1]:
            return prop
        else:
            return ab
    else:
        return ab

In [72]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_gen(example):
    """Label tweet as transparent based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.90:
        if example.xlmroberta_label["labels"][0] == candidate_labels[0]:
            return gen
        else:
            return ab
    else:
        return ab

In [73]:
lfs = [
    missing_bio,
    missing_loc,
    created_2018_2019,
    created_2011_2012,
    bio_keywords,
    # follow_ratio_prop,
    # follow_ratio_gen,
    contain_url,
    contain_mention,
    labeling_sarcasm,
    labeling_hate,
    contain_ent,
    ent_free,
    contain_question,
    loaded_language,
    genuine_language,
    loaded_proppy,
    genuine_proppy,
    loaded_hate,
    loaded_sarcasm,
    gen_hate,
    gen_sarcasm,
    flag_wave,
    repetition,
    distant_supervision_prop,
    distant_supervision_gen,
    slogans,
    reductio,
    exaggeration,
    pronouns,
    xlmroberta_prop,
    xlmroberta_gen,
]

lfs += proppy_lfs # uncomment this line in case of splitted proppy lex

print(f"We have {len(lfs)} LFs used.")

We have 48 LFs used.


In [76]:
applier = PandasLFApplier(lfs)

In [77]:
L_dev = applier.apply(labeled_data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:23<00:00, 21.02it/s]


In [78]:
results = LFAnalysis(L_dev, lfs).lf_summary(labeled_data.label.values)
results.to_json(lfs_analysis_path)
results

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
missing_bio,0,[1],0.076,0.076,0.076,3,35,0.078947
missing_loc,1,[1],0.410,0.410,0.410,20,185,0.097561
created_2018_2019,2,[1],0.084,0.084,0.084,3,39,0.071429
created_2011_2012,3,[0],0.288,0.288,0.288,127,17,0.881944
bio_keywords,4,[0],0.066,0.066,0.066,32,1,0.969697
contain_url,5,[0],0.044,0.044,0.044,21,1,0.954545
contain_mention,6,[1],0.652,0.652,0.652,29,297,0.088957
labeling_sarcasm,7,[1],0.006,0.006,0.006,2,1,0.666667
labeling_hate,8,[1],0.016,0.016,0.016,1,7,0.125000
contain_ent,9,[1],0.396,0.396,0.396,34,164,0.171717


In [79]:
L_train = applier.apply(unlabeled_data)
LFAnalysis(L_train, lfs).lf_summary()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195686/195686 [3:46:48<00:00, 14.38it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
missing_bio,0,[1],0.068758,0.068758,0.068758
missing_loc,1,[1],0.255746,0.255746,0.255746
created_2018_2019,2,[1],0.048532,0.048532,0.048532
created_2011_2012,3,[0],0.371452,0.371452,0.369934
bio_keywords,4,[0],0.326360,0.326360,0.325613
contain_url,5,[0],0.461193,0.461193,0.459593
contain_mention,6,[1],0.478246,0.478246,0.478246
labeling_sarcasm,7,[1],0.007773,0.007773,0.007773
labeling_hate,8,[1],0.014881,0.014881,0.014881
contain_ent,9,[1],0.622962,0.622962,0.622962


In [115]:
L_dev_ids = L_dev.copy()
L_train_ids = L_train.copy()
L_dev_ids= np.append(L_dev_ids, labeled_data.tweetid.to_numpy().reshape(-1, 1), axis=1)
L_train_ids= np.append(L_train_ids, unlabeled_data.tweetid.to_numpy().reshape(-1, 1), axis=1)
L_dev_ids.shape, L_train_ids.shape

((500, 49), (195686, 49))

In [116]:
np.save(lfs_dev_labels_path, L_dev_ids)
np.save(lfs_train_labels_path, L_train_ids)

In [74]:
# loading the saved data

results = pd.read_json(lfs_analysis_path)
L_train = np.load(lfs_train_labels_path, allow_pickle=True)
L_dev = np.load(lfs_dev_labels_path, allow_pickle=True)

In [75]:
results.head(2)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
missing_bio,0,[1],0.076,0.076,0.076,3,35,0.078947
missing_loc,1,[1],0.410,0.410,0.410,20,185,0.097561


In [76]:
L_dev.shape, L_train.shape

((500, 49), (195686, 49))

In [77]:
L_train[:2, -1]

array([503884215260168192, 604045046837465088], dtype=object)

In [78]:
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,544945186909220864,607579193,حمود الداخل,alntafat,"الرياض, المملكة العربية السعودية",مشينا على درب المعزه ونهج العود ومن مدرسه شيخ ...,61366.0,6869.0,2012-06-13 00:00:00,RT @axxa00: توكلت على الله ولا حول ولاقوة الا ...,...,1,0,1,توكلت على الله ولا حول ولاقوة الا بالله استغفر...,1,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9975724816...","{'label': 'not offensive', 'score': 0.99894624...",{'sequence': 'توكلت على الله ولا حول ولاقوة ال...
1,1117798852772401152.0,2673203522.0,None,None,None,None,NaN,NaN,NaT,RT @MOISaudiArabia: الأمير عبدالعزيز بن سعود و...,...,0,1,1,الأمير عبدالعزيز بن سعود وزير الداخلية يرعى حف...,0,13,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9980658888...","{'label': 'not offensive', 'score': 0.99932289...",{'sequence': 'الأمير عبدالعزيز بن سعود وزير ال...


In [79]:
unlabeled_data = unlabeled_data.set_index(unlabeled_data.tweetid).loc[L_train[:, -1]].reset_index(drop=True)
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,503884215260168192,551984787,د. جواهر السيف♛اصايل,Jawaher_ALsaif,"London, England",دكتوراة في العلاقات الدولية ‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏۩...,153445.0,3762.0,2012-04-12 00:00:00,RT @a_alswaiyd: رئيس النصر يطالب بحكام اجانب م...,...,0,1,1,رئيس النصر يطالب بحكام اجانب من الموسم الماضي ...,0,24,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Sarcasm', 'score': 0.8988369703292847}","{'label': 'not offensive', 'score': 0.98927068...",{'sequence': 'رئيس النصر يطالب بحكام اجانب من ...
1,604045046837465088,yaERHdZWPYosxTo2yo5Xli7LyNzwBHrVLaCMO5J+a4=,yaERHdZWPYosxTo2yo5Xli7LyNzwBHrVLaCMO5J+a4=,yaERHdZWPYosxTo2yo5Xli7LyNzwBHrVLaCMO5J+a4=,None,‏استغفر الله العظيم,4212.0,2843.0,2015-01-08 00:00:00,RT @hashtgareed: #عاجل 🔴\nوجد طفل ضايع اليوم ف...,...,3,0,1,عاجل وجد طفل ضايع اليوم في الشعيبة في جلاجل في...,1,20,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9963112473...","{'label': 'not offensive', 'score': 0.99759680...",{'sequence': 'عاجل وجد طفل ضايع اليوم في الشعي...


In [80]:
labeled_data = labeled_data.set_index(labeled_data.tweetid).loc[L_dev[:, -1]].reset_index(drop=True)
labeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label,label
0,1091816593032646656,399347336,أبو عبدالمجيد العميم,AAlOmaim,None,مُتابع لأخبار الطقس وموثق لها في #أرشيف_أبو_عب...,18873.0,93.0,2011-10-27 00:00:00,RT @QX852iVNf3lnJnL85vEdJIwkmC5eZVl7gmFr2ge+G8...,...,0,2,RT +G8=: حاير المجمعة ١٤٤٠هـ الديم والنفيض وال...,0,20,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9976539015...","{'label': 'not offensive', 'score': 0.99891853...",{'sequence': 'RT +G8=: حاير المجمعة ١٤٤٠هـ الد...,0
1,712221860365205504,2237595005,سـمـو الأمـيـر,K__S__A__511,♥♥الـطـآئـفــے♥♥,الطايف ديرة عشق وذكريات ومقادير اجمل مافي مدن ...,454422.0,98015.0,2013-12-22 00:00:00,-\n\nلا أحد يشعر بأحد\n\nهاهي الشمس تحرق نفسها...,...,0,0,- لا أحد يشعر بأحد هاهي الشمس تحرق نفسها من أج...,0,18,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9955314993...","{'label': 'not offensive', 'score': 0.99877494...",{'sequence': '- لا أحد يشعر بأحد هاهي الشمس تح...,0


In [81]:
assert len(unlabeled_data) == len(L_train)
assert len(labeled_data) == len(L_dev)

In [82]:
L_train = L_train[:, :-1].copy()
L_dev = L_dev[:, :-1].copy()

In [87]:
L_dev.shape, L_train.shape

((500, 48), (195686, 48))

In [93]:
L_train = L_train.astype(np.int32)
L_dev = L_dev.astype(np.int32)

In [170]:
# selecting the LFs that maximize the performance of the label model
flag = ~np.array([results["Emp. Acc."] < 0.145, results["Coverage"] >= 0.5]).any(axis=0)
inds = results[flag].j.to_list()
inds.remove(3)
inds.append(8)
inds.append(13)
len(inds)

21

In [171]:
# displaying the names of the LFs chosen
results.iloc[inds, :].index.to_list()

['bio_keywords',
 'contain_url',
 'labeling_sarcasm',
 'contain_ent',
 'contain_question',
 'loaded_language',
 'loaded_hate',
 'loaded_sarcasm',
 'distant_supervision_prop',
 'distant_supervision_gen',
 'slogans',
 'reductio',
 'xlmroberta_prop',
 'xlmroberta_gen',
 'loaded_proppy_factives',
 'loaded_proppy_hedges',
 'loaded_proppy_implicatives',
 'loaded_proppy_bias',
 'loaded_proppy_negative_words',
 'labeling_hate',
 'genuine_language']

In [172]:
L_train_ = L_train[:, inds].copy()
L_dev_ = L_dev[:, inds].copy()

In [173]:
LFAnalysis(L_dev_, np.array(lfs)[inds]).lf_summary(labeled_data.label.values)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
bio_keywords,0,[0],0.066,0.066,0.058,32,1,0.969697
contain_url,1,[0],0.044,0.044,0.026,21,1,0.954545
labeling_sarcasm,2,[1],0.006,0.006,0.004,2,1,0.666667
contain_ent,3,[1],0.396,0.396,0.334,34,164,0.171717
contain_question,4,[1],0.072,0.072,0.054,10,26,0.277778
loaded_language,5,[1],0.192,0.176,0.040,34,62,0.354167
loaded_hate,6,[1],0.034,0.034,0.024,4,13,0.235294
loaded_sarcasm,7,[1],0.008,0.008,0.004,3,1,0.750000
distant_supervision_prop,8,[1],0.048,0.048,0.034,11,13,0.458333
distant_supervision_gen,9,[0],0.126,0.126,0.106,62,1,0.984127


In [174]:
# estimating the class balance in the unlabeled data from the labeled.

w = labeled_data.label.value_counts(normalize=True).to_list()
w

[0.904, 0.096]

In [366]:
# tuning the L2 regularization parameter to maximize the performance of the label model
# we maximize the precision as it is need to surpass 50% for the noise-aware loss

best_score = 0
best_model = None
l2_values = np.arange(0.0, 0.1, 0.01)


metric = "f1"
for l2 in tqdm(l2_values, total=len(l2_values)):
    label_model = LabelModel(cardinality=2, verbose=False, device="cuda")
    label_model.fit(
        L_train=L_train_,
        n_epochs=500,
        l2=l2,
        lr=0.003,
        seed=42,
        class_balance=w,
        progress_bar=False,
        lr_scheduler="linear",
        optimizer="adamax",
        lr_scheduler_config={"warmup_percentage": 0.0, "warmup_unit": "epochs"},
    )
    score = label_model.score(L_dev_, labeled_data.label, metrics=[metric])
    if score[metric] >= best_score:
        best_score = score[metric]
        best_model = label_model

  0%|          | 0/10 [00:00<?, ?it/s]

In [367]:
# displaying the configs that led to the best label model
best_model.train_config

TrainConfig(n_epochs=500, lr=0.003, l2=0.06, optimizer='adamax', optimizer_config=OptimizerConfig(sgd_config=SGDOptimizerConfig(momentum=0.9), adam_config=AdamOptimizerConfig(amsgrad=False, betas=(0.9, 0.999)), adamax_config=AdamaxOptimizerConfig(betas=(0.9, 0.999), eps=1e-08)), lr_scheduler='linear', lr_scheduler_config=LRSchedulerConfig(warmup_steps=0, warmup_unit='epochs', warmup_percentage=0.0, min_lr=0.0, exponential_config=ExponentialLRSchedulerConfig(gamma=0.9), step_config=StepLRSchedulerConfig(gamma=0.9, step_size=5)), prec_init=0.7, seed=42, log_freq=10, mu_eps=None)

Reporting the performance of the label model


In [368]:
best_model.score(
    L_dev_,
    labeled_data.label,
    tie_break_policy="abstain",
    metrics=["accuracy", "precision", "recall", "f1", "f1_macro", "f1_micro"],
)

{'accuracy': 0.924,
 'precision': 0.625,
 'recall': 0.5208333333333334,
 'f1': 0.5681818181818181,
 'f1_macro': 0.7632575757575757,
 'f1_micro': 0.924}

In [369]:
y_true = labeled_data.label
y_pred = best_model.predict(L_dev_)
y_true = y_true[y_pred != -1]
y_pred = y_pred[y_pred != -1]

In [370]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       452
           1       0.62      0.52      0.57        48

    accuracy                           0.92       500
   macro avg       0.79      0.74      0.76       500
weighted avg       0.92      0.92      0.92       500



Saving the weakly labeled dataset


In [371]:
# labeled_data["snorkel"] = best_model.predict(L_dev_)

In [372]:
# saving the datasets
weak_probs = best_model.predict_proba(L_train_)
weak_preds = best_model.predict(L_train_)
unlabeled_data["label"] = weak_preds
unlabeled_data["probs"] = list(weak_probs)
unlabeled_data = unlabeled_data[weak_preds != -1]

In [373]:
unlabeled_data[["tweetid", "text", "label", "probs"]].to_json(weakly_labeled_data_path)